<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM5110-2 Workshop 4.2: Nonlinear FEM with plastic hinges and arclength (notebook 1)

## Euler Beam
In this notebook we consider the geometrically nonlinear behavior of a simply supported beam with two point loads and a plastic hinge. This case has been discussed in the lecture on plasticity in *Stability of Structures*.

<center><img src="https://raw.githubusercontent.com/fmeer/public-files/main/ciem5110/plasticBeam.png" alt="Euler beam" width="700"/></center>



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../../')

from utils import proputils as pu
import main
from names import GlobNames as gn

%matplotlib tk

### Geometrically nonlinear elastic analysis


In [ ]:
plt.close('all')
props = pu.parse_file('elastic.pro')
globdat = main.jive(props)

def plotFu(globdat):
    # make a customized force-displacement plot
    Ftot = globdat['loaddisp']['left']['load']['dx'] + globdat['loaddisp']['mid']['load']['dy']
    umid = globdat['loaddisp']['mid']['disp']['dy']
    plt.figure()
    plt.plot(umid,Ftot,marker='.')
    plt.xlabel('u')
    plt.ylabel('F')
    plt.show()
    
plotFu(globdat)


### Analysis with plastic hinges

We can also perform nonlinear analysis. The input file `nonlin.pro` contains a lot of similar content, but specifies `NonlinModule` instead of `LinBuckModule` to perform a different type of analysis.

In [ ]:
props = pu.parse_file('elastic.pro')
props['model']['frame']['plastic'] = 'True'
props['model']['frame']['Mp'] = '0.4'

plt.close('all')
globdat = main.jive(props)
plotFu(globdat)

### Change $\alpha$ to a lower value

In [ ]:
alpha = 0.5
plt.close('all')
props['model']['neum']['loadIncr'] = str([1-alpha,alpha])
globdat = main.jive(props)
plotFu(globdat)

In [ ]:
# function that runs analysis for given load case
def run_analysis(alpha,props):
    props['model']['neum']['loadIncr'] = str([1-alpha,alpha])
    globdat = main.jive(props)
    F = globdat['loaddisp']['left']['load']['dx'] + globdat['loaddisp']['mid']['load']['dy']
    u = globdat['loaddisp']['mid']['disp']['dy']
    print('done with analysis with alpha ' + str(alpha))
    return F, u

# switch off frameviewmodule
if 'frameview' in props: 
    del props['frameview']    

# run analysis for a number of different alphas
plt.close('all')
for alpha in [0.001,0.01,0.02,0.05,0.1,0.2,0.5,1,1.5]:
    F,u = run_analysis(alpha,props)
    plt.plot(u,F)
plt.show()
